In [1]:
import pandas as pd

In [2]:
matches_firms = pd.read_csv('../data/matches_firms.csv')
matches_funds = pd.read_csv('../data/matches_funds.csv')
matches_owners = pd.read_csv('../data/matches_owners.csv')

In [3]:
matches_firms_funds = pd.merge(matches_firms, matches_funds, how='inner')

In [4]:
# Formatting Data Frame
filter_columns = [
    'form_d_fund_id', 'cik_no_fund', 'fund',
    'form_adv_firm_id', 'crd_no_firm', 'matched_firm', 'firm_confidence',
    'form_adv_fund_id', 'crd_no_fund', 'matched_fund', 'fund_confidence',
]
matches_firms_funds = matches_firms_funds[filter_columns]

In [5]:
matches_firms_funds.head()

,form_d_fund_id,cik_no_fund,fund,form_adv_firm_id,crd_no_firm,matched_firm,firm_confidence,form_adv_fund_id,crd_no_fund,matched_fund,fund_confidence
0,54568,1694312,Ajo Emerging Markets Small Cap Fund,NaN,309921,Capitalworks Emerging Markets Advisor,0.985133,602.0,105545,Ajo Emerging Markets Small Cap Fund,0.000000
1,60070,1685357,Libra Fossil Fuel Free Fund,NaN,281914,Fund,0.927080,11352.0,294197,Libra Fossil Fuel Free Fund,0.000000
2,38363,1679614,Fiam Global Low Volatility Equity Fund,NaN,137649,Global Equity Advisors,0.891444,10035.0,133196,Fiam Global Low Volatility Equity Fund,0.000000
3,43377,1753457,Parian Global Us Fund,20874.0,297279,Parian Global Management,0.819455,20874.0,297279,Parian Global Us Fund Ii,0.263488
4,24565,1450552,Davi Luxury Brand Group,142265.0,306533,Daventry Group,0.983191,3170.0,110885,Capital Group Euro Bond Fund Lux,0.974050


Testing

In [39]:
match_df = matches_firms_funds.iloc[:10]

In [40]:
cols = ['cik_no_fund', 'crd_no_firm', 'crd_no_fund']

match_df = match_df[cols]

In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from ftfy import fix_text

def get_match_df(data_a, data_b):

    def _ngrams(string, n=2):
        string = str(string)
        string = fix_text(string)  # fix text
        n_grams = zip(*[string[i:] for i in range(n)])
        return [''.join(ngram) for ngram in n_grams]

    right_values = data_b.direct_owners.unique()

    vectorizer = TfidfVectorizer(min_df=1, analyzer=_ngrams, lowercase=False)
    tfidf = vectorizer.fit_transform(right_values)
    nbrs = NearestNeighbors(n_neighbors=1, n_jobs=-1).fit(tfidf)

    # column to match against in the data
    left_values = set(data_a.related_partners)  # set used for increased performance

    # matching query
    def _getNearestN(query):
        queryTFIDF_ = vectorizer.transform(query)
        distances, indices = nbrs.kneighbors(queryTFIDF_)
        return distances, indices

    distances, indices = _getNearestN(left_values)

    left_values = list(left_values)  # need to convert back to a list

    matches = []
    for i, j in enumerate(indices):
        temp = [left_values[i], right_values[j][0], distances[i][0]]
        matches.append(temp)

    matches = pd.DataFrame(matches, columns=['related_partners', 'matched_owners', f'owners_confidence'])

    return matches

In [42]:
related_partners = pd.read_pickle('../storage/related_partners.pkl')
direct_owners = pd.read_pickle('../storage/direct_owners.pkl')

result_df = []

for idx, row in match_df.iterrows():

    cik_no_fund, crd_no_firm, crd_no_fund = row.cik_no_fund, row.crd_no_firm, row.crd_no_fund

    table_a = related_partners[related_partners.cik_no_related_partners == cik_no_fund]
    table_b = direct_owners[(direct_owners.crd_no_owners == crd_no_firm) | (direct_owners.crd_no_owners == crd_no_fund)]

    match = get_match_df(table_a, table_b)

    result_df.append(match)

ValueError: Found array with 0 sample(s) (shape=(0, 146)) while a minimum of 1 is required.

In [43]:
result_df[0].head()

IndexError: list index out of range